In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()

bucket = "sunbird-speech"
iam = boto3.client("iam")
role = iam.get_role(RoleName="AmazonSageMaker-ExecutionRole-20210630T164029")['Role']['Arn']
print(role)

arn:aws:iam::224427659724:role/service-role/AmazonSageMaker-ExecutionRole-20210630T164029


In [30]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/iowomugisha/.netrc


True

In [31]:
wandb.sagemaker_auth()

In [10]:
from sagemaker.pytorch import PyTorch

hyperparameters = {
  "model_name_or_path": "indonesian-nlp/wav2vec2-luganda",
  "dataset_name": "dataset/common_voice",
  "dataset_config_name": "lg",
  "dataset_min_filesize": 10000,
  "dataset_min_textlength": 10,
  "output_dir": "output/wav2vec2-common_voice-lg",
  "overwrite_output_dir": True,
  "num_train_epochs": 1,
  "report_to":"wandb",
  "run_name":"sunbird-speech",
  "per_device_eval_batch_size": 2,
  "per_device_train_batch_size": 4,
  "evaluation_strategy": "steps",
  "learning_rate": 1e-4,
  "warmup_steps": 300,
  "fp16": True,
  "freeze_feature_extractor": True,
  "save_steps": 2000,
  "eval_steps": 2000,
  "save_total_limit": 1,
  "logging_steps": 2000,
  "group_by_length": True,
  "feat_proj_dropout": 0.04,
  "layerdrop": 0.041,
  "attention_dropout": 0.094,
  "activation_dropout": 0.055,
  "hidden_dropout": 0.047,
  "mask_time_prob": 0.4,
  "do_train": True,
  "do_eval": True,
  "gradient_accumulation_steps": 2,
  "dataloader_num_workers": 8,
  "train_split_name": "train+validation+other+invalidated"
}

estimator = PyTorch(
    entry_point="run_finetuning.py",
    source_dir='./speech-to-text/',
    role=role,
    py_version="py38",
    framework_version="1.11.0",
    instance_type="ml.p3.2xlarge",
    instance_count=1,
    hyperparameters=hyperparameters,
    base_job_name='speech-to-text-job'
)

In [11]:
common_voice_data = "s3://sunbird-speech/common-voice-lg/common-voice-lg.tar.gz"
estimator.fit(inputs=common_voice_data)

2022-08-26 12:41:20 Starting - Starting the training job...
2022-08-26 12:41:45 Starting - Preparing the instances for trainingProfilerReport-1661517678: InProgress
......
2022-08-26 12:43:03 Downloading - Downloading input data............
2022-08-26 12:45:25 Training - Downloading the training image..............bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-08-26 12:48:10,393 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-08-26 12:48:10,415 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-08-26 12:48:10,422 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-08-26 12:48:10,895 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.8 -m pip install -r requirements.txt
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.6 MB/s eta 0:00:00
━━